In [1]:
from tracing.rl.actions import *
from tracing.rl.a3cmodel import A3CModel
from tracing.rl.rewards import PopupRewardsCalculator
from tracing.rl.environment import Environment
from tracing.rl.actor_learner import ActorLearnerWorker
import tensorflow as tf
import threading

/home/aleksei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
popup_urls = [
    # Choose from two options popups
    'monstervape.com',
    'twistedcigs.com',
    'ecigsejuice.com',
    'vape-fuel.com',
    'powervapes.net',
    'ecigexpress.com',
    'ecigvaporstore.com',
    
    # Subscribe
    'cigarmanor.com',
    
    # Enter date popups
    'thecigarshop.com',
    'cigartowns.com',
    'docssmokeshop.com',
    'enhancedecigs.com',
    'betamorphecigs.com',
    
    # Accept Cookie
    'theglamourshop.com'
]

no_popup_urls = [
    'dixieems.com',
    'firstfitness.com',
    'sandlakedermatology.com',
    'dixieems.com',
    'anabolicwarfare.com',
    'jonessurgical.com',
    'srandd.com']

In [3]:
popup_urls = ['vape-fuel.com']

tf.reset_default_graph()
session = tf.Session()

num_workers = 6

global_model = A3CModel(len(Actions.actions), session = session)
workers = []

for i in range(num_workers):
    env = Environment(PopupRewardsCalculator(), user={}, headless=True)
    workers.append(ActorLearnerWorker("worker-{}".format(i), popup_urls, global_model, env, 1000))

coord = tf.train.Coordinator()
session.run(tf.global_variables_initializer())

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

threads = []
for worker in workers:
    thread = threading.Thread(target=lambda: worker.run())
    thread.start()
    threads.append(thread)

while True:
    time.sleep(30)
    
    rewards = ActorLearnerWorker.step_rewards[:]
    if len(rewards) > 0:
        print('avg_reward:', sum(rewards) / len(rewards))
    plt.plot(np.arange(len(rewards)), rewards)
    plt.xlabel('step')
    plt.ylabel('Total moving reward')
    plt.show()
    
coord.join(threads)



started url http://vape-fuel.com


started url http://vape-fuel.com


started url http://vape-fuel.com


started url http://vape-fuel.com


started url http://vape-fuel.com


started url http://vape-fuel.com


ZeroDivisionError: division by zero

extracted controls: 3
extracted controls: 3
extracted controls: 3
extracted controls: 3
extracted controls: 3
extracted controls: 3
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
got probabilities: [[0.12238529 0.18576527 0.29581252 0.15139307 0.24464387]]
got action: Click
got probabilities: [[0.17444423 0.3711193  0.2

control: Control: link, label: 



Website Age Checker & Age Verification Pop Up Script by AgeVerify.co, values: None, min: None, max = None
image shape: (2854, 612, 4)
got probabilities: [[0.1746829  0.20482464 0.23168088 0.21412864 0.1746829 ]]
got action: InputBYear
controls: 10, covered: 10
reward: -100
controls: 10, covered: 9
control: Control: button, label: I am Under Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
got probabilities: [[0.2 0.2 0.2 0.2 0.2]]
got action: Click
controls: 10, covered: 10
controls: 10, covered: 9
controls: 10, covered: 10
controls: 10, covered: 10
reward: 0
controls: 10, covered: 3
reward: -100
batch_size:  2
controls: 10, covered: 9
reward: -100
batch_size:  3
controls: 10, covered: 10
control: Control: link, label: 



Website Age Checker & Age Verification Pop Up Script by AgeVerify.co, values: None, min: None, max = None
image shape: (2854, 612, 4)
-295.02


started url http://vape-fuel.com
got probabilities: [[0.169751

reward: -100
control: Control: button, label: I am Under Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
got probabilities: [[0.1706642  0.30222496 0.1706642  0.18578248 0.1706642 ]]
got action: Click
controls: 10, covered: 9
controls: 10, covered: 9
controls: 10, covered: 10
controls: 10, covered: 9
reward: 0
controls: 10, covered: 3
reward: -100
batch_size:  2
controls: 10, covered: 9
reward: -100
batch_size:  3
-295.02


started url http://vape-fuel.com
control: Control: button, label: I am Under Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
extracted controls: 3
got probabilities: [[0.17481157 0.20434138 0.23216751 0.21386795 0.17481157]]
got action: Do Nothing
-292.0698


started url http://vape-fuel.com
control: Control: button, label: I am Of/Over Legal Age, values: None, min: None, max = None
image shape: (2854, 612, 4)
got probabilities: [[0.12089798 0.18311441 0.2973167  0.15186566 0.24680522]]
got action: InputBMonth
ex

In [ ]:
# Test Environment
from tracing.selenium_utils.common import *

env = Environment(PopupRewardsCalculator(), user={}, headless=False)

with env:
    env.start('enhancedecigs.com')
    env.driver.set_script_timeout(100)

    ctrls = env.get_controls()
    assert len(ctrls) == 6

    dayCtrl = ctrls[0]
    monthCtrl = ctrls[1]
    yearCtrl = ctrls[2]
    check = ctrls[3]
    enter = ctrls[5]

    assert env.apply_action(dayCtrl, InputBDay()) == 0
    assert env.apply_action(monthCtrl, InputBMonth()) == 0
    assert env.apply_action(yearCtrl, InputBYear()) == 0
    assert env.apply_action(check, Click()) == 0
    assert env.apply_action(enter, Click()) == 100

    assert env.calc_final_reward() == 0


In [ ]:
# Test Popups Detecting

import time

rewards = PopupRewardsCalculator()

driver = create_chrome_driver()

for url in no_popup_urls:
    print('url: ', url)
    driver.get('http://' + url)
    time.sleep(2)
    assert not rewards.is_popup_exists(driver)

for url in popup_urls:
    print('url: ', url)
    driver.get('http://' + url)
    time.sleep(2)
    assert rewards.is_popup_exists(driver)

driver.quit()    